In [50]:
import os
from google.colab import userdata
import requests
import hashlib
import time
import pandas as pd
from pprint import pprint as pp

public_key=userdata.get('public_key')
private_key=userdata.get('private_key')
print(private_key)

ts = str(time.time())
to_hash = ts + str(private_key) + str(public_key)
hash_md5 = hashlib.md5(to_hash.encode('utf-8')).hexdigest()

characters = "https://gateway.marvel.com/v1/public/characters"
comics = "https://gateway.marvel.com/v1/public/comics"

params = {
    "ts": ts,
    "apikey": public_key,
    "hash": hash_md5,
    "limit": 100
}




c3795dc106f083d2611a7cbef26353930f285f45


# FUNCTIONS

In [45]:
def get_data(url, public_key, private_key, limit_num, offset_num):
  ts = str(time.time())
  to_hash = ts + str(private_key) + str(public_key)
  hash_md5 = hashlib.md5(to_hash.encode('utf-8')).hexdigest()

  params = {
    "ts": ts,
    "apikey": public_key,
    "hash": hash_md5,
    "limit": limit_num,
    "offset": offset_num
  }

  response = requests.get(url, params=params)
  data = response.json()
  df = pd.DataFrame(data)
  df_normalizado = pd.json_normalize(df["data"]["results"])
  return df_normalizado

In [81]:
# TO DO: Adicionar esse total no GET ALL
data["data"]["total"]

64269

# CHARACTERS

In [59]:
limit_num = 100
offset_num = 0

personagens = get_data(characters, public_key, private_key, limit_num, offset_num)

In [60]:
personagens.columns

Index(['id', 'name', 'description', 'modified', 'resourceURI', 'urls',
       'thumbnail.path', 'thumbnail.extension', 'comics.available',
       'comics.collectionURI', 'comics.items', 'comics.returned',
       'series.available', 'series.collectionURI', 'series.items',
       'series.returned', 'stories.available', 'stories.collectionURI',
       'stories.items', 'stories.returned', 'events.available',
       'events.collectionURI', 'events.items', 'events.returned'],
      dtype='object')

In [61]:
personagens.head()

,id,name,description,modified,resourceURI,urls,thumbnail.path,thumbnail.extension,comics.available,comics.collectionURI,...,series.items,series.returned,stories.available,stories.collectionURI,stories.items,stories.returned,events.available,events.collectionURI,events.items,events.returned
0,1011334,3-D Man,,2014-04-29T14:18:17+0000,https://gateway.marvel.com/v1/public/character...,"[{'type': 'detail', 'url': 'http://marvel.com/...",http://i.annihil.us/u/prod/marvel/i/mg/c/e0/53...,jpg,12,https://gateway.marvel.com/v1/public/character...,...,[{'resourceURI': 'https://gateway.marvel.com/v...,3,21,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,20,1,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,1
1,1017100,A-Bomb (HAS),Rick Jones has been Hulk's best bud since day ...,2013-09-18T15:54:04+0000,https://gateway.marvel.com/v1/public/character...,"[{'type': 'detail', 'url': 'http://marvel.com/...",http://i.annihil.us/u/prod/marvel/i/mg/3/20/52...,jpg,4,https://gateway.marvel.com/v1/public/character...,...,[{'resourceURI': 'https://gateway.marvel.com/v...,2,7,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,7,0,https://gateway.marvel.com/v1/public/character...,[],0
2,1009144,A.I.M.,AIM is a terrorist organization bent on destro...,2013-10-17T14:41:30+0000,https://gateway.marvel.com/v1/public/character...,"[{'type': 'detail', 'url': 'http://marvel.com/...",http://i.annihil.us/u/prod/marvel/i/mg/6/20/52...,jpg,53,https://gateway.marvel.com/v1/public/character...,...,[{'resourceURI': 'https://gateway.marvel.com/v...,20,61,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,20,0,https://gateway.marvel.com/v1/public/character...,[],0
3,1010699,Aaron Stack,,1970-01-01T00:00:00+0000,https://gateway.marvel.com/v1/public/character...,"[{'type': 'detail', 'url': 'http://marvel.com/...",http://i.annihil.us/u/prod/marvel/i/mg/b/40/im...,jpg,14,https://gateway.marvel.com/v1/public/character...,...,[{'resourceURI': 'https://gateway.marvel.com/v...,3,28,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,20,0,https://gateway.marvel.com/v1/public/character...,[],0
4,1009146,Abomination (Emil Blonsky),"Formerly known as Emil Blonsky, a spy of Sovie...",2014-06-27T19:39:07+0000,https://gateway.marvel.com/v1/public/character...,"[{'type': 'detail', 'url': 'http://marvel.com/...",http://i.annihil.us/u/prod/marvel/i/mg/9/50/4c...,jpg,55,https://gateway.marvel.com/v1/public/character...,...,[{'resourceURI': 'https://gateway.marvel.com/v...,20,68,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,20,1,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,1


# COMICS

In [62]:
limit_num = 100
offset_num = 0

quadrinhos = get_data(comics, public_key, private_key, limit_num, offset_num)

In [63]:
quadrinhos.columns

Index(['id', 'digitalId', 'title', 'issueNumber', 'variantDescription',
       'description', 'modified', 'isbn', 'upc', 'diamondCode', 'ean', 'issn',
       'format', 'pageCount', 'textObjects', 'resourceURI', 'urls', 'variants',
       'collections', 'collectedIssues', 'dates', 'prices', 'images',
       'series.resourceURI', 'series.name', 'thumbnail.path',
       'thumbnail.extension', 'creators.available', 'creators.collectionURI',
       'creators.items', 'creators.returned', 'characters.available',
       'characters.collectionURI', 'characters.items', 'characters.returned',
       'stories.available', 'stories.collectionURI', 'stories.items',
       'stories.returned', 'events.available', 'events.collectionURI',
       'events.items', 'events.returned'],
      dtype='object')

In [64]:
quadrinhos.head()

,id,digitalId,title,issueNumber,variantDescription,description,modified,isbn,upc,diamondCode,...,characters.items,characters.returned,stories.available,stories.collectionURI,stories.items,stories.returned,events.available,events.collectionURI,events.items,events.returned
0,82967,0,Marvel Previews (2017),0,,,2019-11-07T08:46:15+0000,,75960608839302811,,...,[],0,2,https://gateway.marvel.com/v1/public/comics/82...,[{'resourceURI': 'https://gateway.marvel.com/v...,2,0,https://gateway.marvel.com/v1/public/comics/82...,[],0
1,82965,0,Marvel Previews (2017),0,,,2019-08-21T17:11:27+0000,,75960608839302611,JUL190068,...,[],0,2,https://gateway.marvel.com/v1/public/comics/82...,[{'resourceURI': 'https://gateway.marvel.com/v...,2,0,https://gateway.marvel.com/v1/public/comics/82...,[],0
2,82970,52952,Marvel Previews (2017),0,,,2020-02-07T09:35:32+0000,,75960608839303111,,...,[],0,2,https://gateway.marvel.com/v1/public/comics/82...,[{'resourceURI': 'https://gateway.marvel.com/v...,2,0,https://gateway.marvel.com/v1/public/comics/82...,[],0
3,61137,0,The Punisher (2016) #5 (Cosplay Variant),5,Cosplay Variant,None,2016-09-01T12:40:16+0000,,75960608413500521,,...,[],0,2,https://gateway.marvel.com/v1/public/comics/61...,[{'resourceURI': 'https://gateway.marvel.com/v...,2,0,https://gateway.marvel.com/v1/public/comics/61...,[],0
4,37497,0,MARVELS (1994) #1,1,,None,2014-05-08T12:18:55+0000,,,,...,[],0,0,https://gateway.marvel.com/v1/public/comics/37...,[],0,0,https://gateway.marvel.com/v1/public/comics/37...,[],0


# GET ALL

## GET ALL CHARACTERS

In [68]:

all_personagens = pd.DataFrame(columns=personagens.columns)

offset_get_all = 0
batch_personagens = get_data(characters, public_key, private_key, limit_num, offset_get_all)
all_personagens = pd.concat([all_personagens, batch_personagens], ignore_index=True)

,id,name,description,modified,resourceURI,urls,thumbnail.path,thumbnail.extension,comics.available,comics.collectionURI,...,series.items,series.returned,stories.available,stories.collectionURI,stories.items,stories.returned,events.available,events.collectionURI,events.items,events.returned


In [69]:
personagens1 = get_data(characters, public_key, private_key, limit_num, 100)
personagens2 = get_data(characters, public_key, private_key, limit_num, 200)

In [76]:
personagens3 = pd.concat([all_personagens, personagens1], ignore_index=True)
personagens3 = pd.concat([personagens3,personagens2], ignore_index=True)

In [78]:
personagens3

,id,name,description,modified,resourceURI,urls,thumbnail.path,thumbnail.extension,comics.available,comics.collectionURI,...,series.items,series.returned,stories.available,stories.collectionURI,stories.items,stories.returned,events.available,events.collectionURI,events.items,events.returned
0,1009175,Beast,,2016-07-05T14:30:45+0000,https://gateway.marvel.com/v1/public/character...,"[{'type': 'detail', 'url': 'http://marvel.com/...",http://i.annihil.us/u/prod/marvel/i/mg/2/80/51...,jpg,833,https://gateway.marvel.com/v1/public/character...,...,[{'resourceURI': 'https://gateway.marvel.com/v...,20,930,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,20,30,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,20
1,1010909,Beast (Earth-311),,1970-01-01T00:00:00+0000,https://gateway.marvel.com/v1/public/character...,"[{'type': 'detail', 'url': 'http://marvel.com/...",http://i.annihil.us/u/prod/marvel/i/mg/5/a0/4c...,jpg,0,https://gateway.marvel.com/v1/public/character...,...,[],0,0,https://gateway.marvel.com/v1/public/character...,[],0,0,https://gateway.marvel.com/v1/public/character...,[],0
2,1010908,Beast (Ultimate),,2014-03-05T13:19:55+0000,https://gateway.marvel.com/v1/public/character...,"[{'type': 'detail', 'url': 'http://marvel.com/...",http://i.annihil.us/u/prod/marvel/i/mg/5/d0/53...,jpg,48,https://gateway.marvel.com/v1/public/character...,...,[{'resourceURI': 'https://gateway.marvel.com/v...,9,90,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,20,0,https://gateway.marvel.com/v1/public/character...,[],0
3,1009176,Becatron,,1970-01-01T00:00:00+0000,https://gateway.marvel.com/v1/public/character...,"[{'type': 'detail', 'url': 'http://marvel.com/...",http://i.annihil.us/u/prod/marvel/i/mg/b/40/im...,jpg,0,https://gateway.marvel.com/v1/public/character...,...,[],0,0,https://gateway.marvel.com/v1/public/character...,[],0,0,https://gateway.marvel.com/v1/public/character...,[],0
4,1009177,Bedlam,,1970-01-01T00:00:00+0000,https://gateway.marvel.com/v1/public/character...,"[{'type': 'detail', 'url': 'http://marvel.com/...",http://i.annihil.us/u/prod/marvel/i/mg/b/40/im...,jpg,8,https://gateway.marvel.com/v1/public/character...,...,[{'resourceURI': 'https://gateway.marvel.com/v...,6,7,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,7,0,https://gateway.marvel.com/v1/public/character...,[],0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,1011333,Dakota North,,2011-06-16T16:58:57+0000,https://gateway.marvel.com/v1/public/character...,"[{'type': 'detail', 'url': 'http://marvel.com/...",http://i.annihil.us/u/prod/marvel/i/mg/7/d0/4c...,jpg,21,https://gateway.marvel.com/v1/public/character...,...,[{'resourceURI': 'https://gateway.marvel.com/v...,6,22,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,20,1,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,1
196,1009259,Damage Control,,2016-09-06T11:36:18+0000,https://gateway.marvel.com/v1/public/character...,"[{'type': 'detail', 'url': 'http://marvel.com/...",http://i.annihil.us/u/prod/marvel/i/mg/5/00/4c...,jpg,16,https://gateway.marvel.com/v1/public/character...,...,[{'resourceURI': 'https://gateway.marvel.com/v...,6,18,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,18,0,https://gateway.marvel.com/v1/public/character...,[],0
197,1009453,Dani Moonstar,,2013-08-06T23:17:59+0000,https://gateway.marvel.com/v1/public/character...,"[{'type': 'detail', 'url': 'http://marvel.com/...",http://i.annihil.us/u/prod/marvel/i/mg/6/20/4c...,jpg,78,https://gateway.marvel.com/v1/public/character...,...,[{'resourceURI': 'https://gateway.marvel.com/v...,20,98,https://gateway.marvel.com/v1/public/character...,[{'resource